# Contribution Tutorial Test Suite

This test suite demonstrates the permutations of the following:
- Node Type: 
    - local runtime created
    - weave.save(d)
    - weave.publish(ed)
- Node Compisition: 
    - Primitive Python Value (int)
    - Op returning primitive (make_number)
    - Custom Type (Fraction)
    - Op Consuming Custom Producing Primitive (to_float)
    - Op COnsiming Custom Producting Custom (reduce)

In [ ]:
# !rm -rf /tmp/local-artifacts

In [ ]:
IS_CI = False

In [ ]:
import weave
import random

# Randomness allows us to avoid cache hits
def rand_int():
    return int(random.random() * 1e10)

def run_test(label, test_case):
    print(f"Test: Runtime, {label}")
    print(f"{weave.use(test_case())=}")
    weave.show(test_case())

    print(f"Test: Local Save, {label}")
    print(f"{weave.use(weave.save(test_case()))=}")
    weave.show(weave.save(test_case()))
    
    # CI does not support publishing
    if not IS_CI:
        print(f"Test: Remote Publish, {label}")
        print(f"{weave.use(weave.publish(test_case()))=}")
        weave.show(weave.publish(test_case()))

In [ ]:
run_test("Primitive Type", rand_int)

In [ ]:
@weave.op(render_info={'type': 'function'})
def make_number(val: int) -> int:
    return val

In [ ]:
def make_number_op():
    return make_number(rand_int())
run_test("Op Returning Primitive Type", make_number_op)

In [ ]:
import math

@weave.type()
class Fraction:
    numerator: int
    denominator: int

    @weave.op()
    def as_float(self) -> float:
        return self.numerator / self.denominator
    
    @weave.op()
    def reduce(self) -> Fraction:
        gcd = math.gcd(self.numerator, self.denominator)
        return Fraction(int(self.numerator / gcd), int(self.denominator / gcd))
    
    # UGG - this should be automatic!
    @weave.op()
    def get_numerator(self) -> int:
        return self.numerator
    
    # UGG - this should be automatic!
    @weave.op()
    def get_denominator(self) -> int:
        return self.denominator
    
@weave.op()
def show_fraction(frac: weave.Node[Fraction]) -> weave.panels.Group:
    return weave.panels.Group(
        items=[
            weave.panels.LabeledItem(label="numerator", item=frac.get_numerator()),
            weave.panels.LabeledItem(label="denominator", item=frac.get_denominator()),
            weave.panels.LabeledItem(label="fraction", item=frac.as_float()),
        ],
    ) 

def make_fraction():
    return Fraction(rand_int(), rand_int())

In [ ]:
run_test("Custom Type", make_fraction)

In [ ]:
def make_custom_chained():
    return make_fraction().as_float()

run_test("Custom Type to Chained Op to Primitive", make_custom_chained)

In [ ]:
def make_custom_chained_to_custom():
    return make_fraction().reduce()

run_test("Custom Type to Chained Op to Custom Type", make_custom_chained_to_custom)